In [25]:
# simple LSTM
import numpy as np
import pandas as pd
from string import punctuation
import torch
from nltk.tokenize import word_tokenize

In [10]:
with open("../data/sentiment_labelled_sentences/sentiment.txt") as f:
    reviews = f.read()

In [15]:
data =  pd.DataFrame([review.split("\t") for review in reviews.split("\n")])
data.columns = ["review", 'sentiment']
data.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [30]:
def split_words_reviews(data):
    text = data.review.tolist()
    clean_text = []
    for t in text:
        clean_text.append(t.translate(str.maketrans('', '', punctuation)).lower().rstrip())
    tokenized = [word_tokenize(x) for x in clean_text]
    all_text  = []
    for tokens in tokenized:
        for to in tokens:
            all_text.append(to)
    return tokenized, set(all_text)
                
    

In [31]:
reviews, vocab = split_words_reviews(data)

In [33]:
# assemble lookuo dictionaires

In [45]:
def create_dictionaries(words):

    word_to_int_dict = {w: i+1 for i, w in enumerate(words)}
    int_to_word_dict = {i: w for w, i in word_to_int_dict.items()}
    return word_to_int_dict, int_to_word_dict
    

In [54]:
word_to_int_dict, int_to_word_dict = create_dictionaries(vocab)
print(word_to_int_dict['imperial'])
int_to_word_dict[2]

2


'imperial'

In [57]:
# decide on padding (max & mean length)
lengths = [len(x) for x in reviews]
print(np.max(lengths), np.median(lengths), np.mean(lengths))

70 10.0 11.783666666666667


In [58]:
input_size = 50


In [63]:
def pad_text(tokenized_reviews, seq_length):
    reviews = []
    for review in tokenized_reviews:
        if len(review) >= seq_length:
            reviews.append(review[:seq_length])
        else:
            reviews.append(['']*(seq_length - len(review)) + review )
    return np.array(reviews)

In [64]:
padded_sentences = pad_text(reviews, seq_length = 50)

In [66]:
padded_sentences[0]

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', 'a', 'very', 'very', 'very', 'slowmoving', 'aimless',
       'movie', 'about', 'a', 'distressed', 'drifting', 'young', 'man'],
      dtype='<U33')

In [67]:
# add empty token to dictionary
word_to_int_dict[''] = 0
int_to_word_dict[0] = ''

In [68]:
# now encode sentences
encoded_sentences = np.array([[word_to_int_dict[word] for word in review ] for review in padded_sentences])

In [70]:
encoded_sentences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 1952, 3552, 3552, 3552, 4001, 2309, 4762,
       1641, 1952, 3164, 5012, 2445, 2118])

In [71]:
# model architecture

In [72]:
from torch import nn

In [85]:
class SentimentLSTM(nn.Module):
    def __init__(self, n_vocab, n_embed, n_hidden, n_output, n_layers, drop_p=0.8):
        super().__init__()
        self.n_vocab = n_vocab
        self.n_embed = n_embed
        self.n_hidden = n_hidden
        
        self.embedding = nn.Embedding(n_vocab, n_embed)
        self.lstm = nn.LSTM(n_embed, n_hidden, n_layers, batch_first = True, dropout = drop_p)
        self.dropout = nn.Dropout(drop_p)
        seld.fc = nn.Linear(n_hidden, n_output)
        self.sigmoid = nn.Sigmoid()
        
    
    def forward(self, input_words):
        
        embedded_words = self.embedding(input_words)
        lstm_out, h = self.lstm(embedded_words)
        lstm_out = self.dropout(lstm_out)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden)
        fc_out = self.fc(lstm_out)
        sigmoid_out = self.sigmoid(fc_out)
        sigmoid_out = sigmoid_out.view(-1, batch_size, -1)
        sigmoid_last - sigmoid_out[:, -1]
        return sigmoid_last, h
    
    def init_hidden(self, batch_size):
        device = "cpu"
        weights = next(self.paramaters()).data
        h = (weights.new(self.n_layers, 
                         batch_size,
                         self.n_hidden
                        ).zero_().to(device),
             weights.new(self.n_layers, 
                         batch_size,
                         self.n_hidden
                        ).zero_().to(device),
            )
        return h
    

In [86]:
n_vocab = len(word_to_int_dict)
n_embed = 50
n_hidden = 100
n_output = 1
n_layers = 2

net = SentimentLSTM(n_vocab, n_embed, n_hidden, n_output, n_layers)

NameError: name 'seld' is not defined